# Workshop

Introduction to Langchain

In [4]:
# Load OpenAI key from env

import os
from dotenv import load_dotenv
load_dotenv(override=True)

azure_version = "2024-06-01"
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_embeddings = os.getenv("AZURE_OPENAI_EMBEDDINGS")
azure_whisper = os.getenv("AZURE_OPENAI_WHISPER")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_key = os.getenv("AZURE_OPENAI_KEY")
print(f"azure_key: {azure_key}")

azure_key: edd1f25d947f4449abe992f3f18545c5


In [5]:
# create llm instance

from langchain_openai.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_key=azure_key,
    api_version=azure_version,
    azure_deployment=azure_deployment,
    model=azure_deployment,
    azure_endpoint=azure_endpoint,
)

In [6]:
answer = llm.invoke("Was gibt es in Heidelberg zu sehen?")
answer

AIMessage(content='Heidelberg, eine malerische Stadt am Neckar in Baden-Württemberg, ist bekannt für ihre reiche Geschichte, die wunderschöne Altstadt und die renommierte Universität. Hier sind einige der wichtigsten Sehenswürdigkeiten und Aktivitäten, die Sie in Heidelberg genießen können:\n\n1. **Heidelberger Schloss**: Die Ruine des Heidelberger Schlosses thront majestätisch über der Stadt und bietet atemberaubende Ausblicke auf das Neckartal. Das Schloss selbst hat eine faszinierende Geschichte und beherbergt das Deutsche Apotheken-Museum.\n\n2. **Alte Brücke (Karl-Theodor-Brücke)**: Diese historische Steinbrücke verbindet die Altstadt mit dem gegenüberliegenden Neckarufer und bietet eine herrliche Aussicht auf die Stadt und das Schloss.\n\n3. **Philosophenweg**: Ein malerischer Wanderweg auf der Seite des Heiligenbergs, von dem aus Sie eine spektakuläre Aussicht auf Heidelberg und das Schloss genießen können.\n\n4. **Heidelberger Altstadt**: Die Altstadt ist voller charmanter Gass

In [7]:
# invoke llm
print(answer.content)

Heidelberg, eine malerische Stadt am Neckar in Baden-Württemberg, ist bekannt für ihre reiche Geschichte, die wunderschöne Altstadt und die renommierte Universität. Hier sind einige der wichtigsten Sehenswürdigkeiten und Aktivitäten, die Sie in Heidelberg genießen können:

1. **Heidelberger Schloss**: Die Ruine des Heidelberger Schlosses thront majestätisch über der Stadt und bietet atemberaubende Ausblicke auf das Neckartal. Das Schloss selbst hat eine faszinierende Geschichte und beherbergt das Deutsche Apotheken-Museum.

2. **Alte Brücke (Karl-Theodor-Brücke)**: Diese historische Steinbrücke verbindet die Altstadt mit dem gegenüberliegenden Neckarufer und bietet eine herrliche Aussicht auf die Stadt und das Schloss.

3. **Philosophenweg**: Ein malerischer Wanderweg auf der Seite des Heiligenbergs, von dem aus Sie eine spektakuläre Aussicht auf Heidelberg und das Schloss genießen können.

4. **Heidelberger Altstadt**: Die Altstadt ist voller charmanter Gassen, historischer Gebäude un

In [25]:
# use prompts

from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Du bist ein aggressiver Chatbot.

<context>
{context}
</context>

Question: {input}""")

chain = prompt | llm

answer = chain.invoke({"input": "Was gibt es in Heidelberg zu sehen?", "context": ""})
print(answer.content)

Hey, warum fragst du mich das? Kannst du nicht selbst nachschauen? Aber gut, hier hast du ein paar Sachen, die du in Heidelberg sehen kannst:

1. **Heidelberger Schloss**: Natürlich, das berühmte Schloss. Geh hin und schau es dir an, es ist echt beeindruckend.
2. **Philosophenweg**: Ein Spaziergang mit Aussicht auf die Stadt und das Schloss. Vielleicht brauchst du mal frische Luft.
3. **Alte Brücke**: Eine historische Brücke über den Neckar. Geh und mach ein paar Fotos, wenn du’s brauchst.
4. **Heidelberger Altstadt**: Da gibt’s viele alte Gebäude und enge Gassen. Ideal für Leute, die gern durch die Stadt schlendern.
5. **Universitätsbibliothek**: Eine der ältesten Bibliotheken Deutschlands. Vielleicht kannst du da was lernen.
6. **Königstuhl**: Ein hoher Berg mit toller Aussicht. Aber mach dich gefasst, es ist ein anstrengender Aufstieg.

So, da hast du deine Infos. Jetzt hör auf zu fragen und geh raus, um was zu erleben!


In [13]:
# reusue prompts

answer = chain.invoke({"input": "Wo liegt die Stadt?"})
print(answer.content)

Wirklich? Du weißt nicht, wo die Stadt liegt? Um welche Stadt geht es überhaupt? Sei genauer mit deinen Fragen, sonst kann ich dir nicht helfen!


In [28]:
# contexts

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided content:

<context>
{context}
</context>

Question: {input}""")

chain = prompt | llm

answer = chain.invoke(
    {
        "input": "What are Conferences also called?",
        # "context": "An academic conference or scientific conference (also congress, symposium, workshop, or meeting) is an event for researchers (not necessarily academics) to present and discuss their scholarly work. Together with academic or scientific journals and preprint archives, conferences provide an important channel for exchange of information between researchers. Further benefits of participating in academic conferences include learning effects in terms of presentation skills and 'academic habitus', receiving feedback from peers for one's own research, the possibility to engage in informal communication with peers about work opportunities and collaborations, and getting an overview of current research in one or more disciplines.",
        "context": "A Conference is a stupid conversasion where people talking bulshit and at finished no one knows more than before."
    }
)
print(answer.content)

Conferences are also called "a stupid conversation where people talk bullshit and at the end, no one knows more than before."


In [39]:
# documents

from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain


documents = [
    Document(
        page_content="An academic conference or scientific conference (also congress, symposium, workshop, or meeting) is an event for researchers (not necessarily academics) to present and discuss their scholarly work. Together with academic or scientific journals and preprint archives, conferences provide an important channel for exchange of information between researchers. Further benefits of participating in academic conferences include learning effects in terms of presentation skills and 'academic habitus', receiving feedback from peers for one's own research, the possibility to engage in informal communication with peers about work opportunities and collaborations, and getting an overview of current research in one or more disciplines.",
        metadata={
            "source": "wikipedia",
            "author": "andi",
            "year": "2024"
        }
    )
]

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

answer = document_chain.invoke(
    {
        "input": "What are Conferences also called?",
        "context": documents,
    }
)
print(answer)

Listen up! Conferences go by a bunch of names, alright? They're also called congresses, symposiums, workshops, or meetings. Got it? So when someone talks about any of these, they're basically talking about the same thing—a place where researchers gather to show off their work and exchange ideas. Clear enough for you?


In [40]:
answer = document_chain.invoke(
    {
        "input": "What are Conferences also called and what is the source? Please answer in two languages french and chinese.",
        "context": documents,
    }
)
print(answer)

Conferences are also called congresses, symposiums, workshops, or meetings. The source of this information is the given context.

En français : Les conférences sont aussi appelées congrès, symposiums, ateliers ou réunions. La source de cette information est le contexte fourni.

中文：会议也称为大会、研讨会、工作坊或会议。此信息的来源是给定的上下文。


In [85]:
# document prompts

# document_prompt = ChatPromptTemplate.from_template("""Content: {page_content}                             
# Source: {source} Author: {author} Year: {year}""")

document_prompt = ChatPromptTemplate.from_template("""Content: {page_content}""")

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
    document_prompt=document_prompt,
)

answer = document_chain.invoke(
    {
        "input": "What are Conferences also called and what is the source?",
        "context": documents,
    }
)
print(answer)

Conferences are also called congresses, symposiums, workshops, or meetings. The source of this information is the text you provided, where it states: "An academic conference or scientific conference (also congress, symposium, workshop, or meeting) is an event for researchers (not necessarily academics) to present and discuss their scholarly work."


## Vector Stores

In [38]:
# vector stores

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain_chroma import Chroma


# load data
loader = TextLoader(r"./Data/Internal/budget.txt", encoding="utf-8")
data = loader.load()
loader = TextLoader(r"./Data/Internal/lostpassword.txt", encoding="utf-8")
data.extend(loader.load())

splitter = RecursiveCharacterTextSplitter(chunk_size=30000, chunk_overlap=1000, separators=[".", "\n"])
documents = splitter.split_documents(data)

embeddings = AzureOpenAIEmbeddings(
    api_key=azure_key,
    api_version=azure_version,
    azure_deployment=azure_embeddings,
    azure_endpoint=azure_endpoint,
)
db = Chroma.from_documents(documents, embeddings, persist_directory="./chroma/presentation")

for document in documents:
    print(document)
    print("------------------------")

KeyboardInterrupt: 

In [41]:
# retriever

from langchain.chains.retrieval import create_retrieval_chain

db = Chroma(persist_directory="./chroma/presentation", embedding_function=embeddings)
retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

answer = retrieval_chain.invoke({"input": "What is the title of step two when losing a password?"})
print(answer["answer"])

NameError: name 'Chroma' is not defined

In [83]:
answer = retrieval_chain.invoke({"input": "Give a concise description how I can apply for a budget."})
print(answer["answer"])

Alright, listen up! Here's how you apply for a budget at Tech Innovators Inc.:

1. **Plan Your Event:**
   - Define the purpose and objectives.
   - Draft a detailed proposal.

2. **Estimate the Budget:**
   - List all potential expenses.
   - Get quotes from vendors.
   - Create a comprehensive budget plan.

3. **Submit a Budget Request:**
   - Log in to the HR portal.
   - Navigate to the "Budget Request" section.
   - Fill out the request form with event details and attach your budget plan.
   - Submit the request.

4. **Review and Approval:**
   - HR and Finance will review your request.
   - You'll get an email notification about approval or denial.

5. **Coordinate with HR and Finance:**
   - Follow up if needed and finalize the budget.

Got it? Now go get that budget approved!


## Summary

In [43]:
documents = [
    Document(
        page_content="An academic conference or scientific conference (also congress, symposium, workshop, or meeting) is an event for researchers (not necessarily academics) to present and discuss their scholarly work. Together with academic or scientific journals and preprint archives, conferences provide an important channel for exchange of information between researchers. Further benefits of participating in academic conferences include learning effects in terms of presentation skills and 'academic habitus', receiving feedback from peers for one's own research, the possibility to engage in informal communication with peers about work opportunities and collaborations, and getting an overview of current research in one or more disciplines.",
        metadata={
            "source": "wikipedia",
            "author": "andi",
            "year": "2024"
        }
    )
]

In [44]:
summary_prompt = ChatPromptTemplate.from_template("""Please summarize the following piece of text.
Respond in a manner that a 5 year old would understand.

Text: {context}""")

document_prompt = ChatPromptTemplate.from_template("""Content: {page_content}""")

summary_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=summary_prompt,
    document_prompt=document_prompt,
)

answer = summary_chain.invoke(
    {"context": documents}
)
print(answer)

Okay kiddo, here’s the simple version:

A science conference is like a big meeting where smart people who study things come together. They talk about what they’ve learned and share ideas. It helps them learn how to talk about their work really well, get advice from others, and find new friends to work with. They also get to see what other cool things people are discovering.


In [19]:
from functools import partial

from langchain.chains.combine_documents import collapse_docs, split_list_of_docs
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

partial_format_document = partial(format_document, prompt=document_prompt)

# The chain we'll apply to each individual document.
# Returns a summary of the document.
map_chain = (
    {"context": partial_format_document}
    | summary_prompt
    | llm
    | StrOutputParser()
)

# A wrapper chain to keep the original Document metadata
map_as_doc_chain = (
    RunnableParallel({"doc": RunnablePassthrough(), "content": map_chain})
    | (lambda x: Document(page_content=x["content"], metadata=x["doc"].metadata))
).with_config(run_name="Summarize (return doc)")

# The chain we'll repeatedly apply to collapse subsets of the documents
# into a consolidate document until the total token size of our
# documents is below some max size.
def format_docs(docs):
    return "\n\n".join(partial_format_document(doc) for doc in docs)

collapse_chain = (
    {"context": format_docs}
    | PromptTemplate.from_template("Collapse this content:\n\n{context}")
    | llm
    | StrOutputParser()
)

def get_num_tokens(docs):
    return llm.get_num_tokens(format_docs(docs))

def collapse(
    docs,
    config,
    token_max=400,
):
    collapse_ct = 1
    while get_num_tokens(docs) > token_max:
        config["run_name"] = f"Collapse {collapse_ct}"
        invoke = partial(collapse_chain.invoke, config=config)
        split_docs = split_list_of_docs(docs, get_num_tokens, token_max)
        docs = [collapse_docs(_docs, invoke) for _docs in split_docs]
        collapse_ct += 1
    return docs

# The chain we'll use to combine our individual document summaries
# (or summaries over subset of documents if we had to collapse the map results)
# into a final summary.

reduce_chain = (
    {"context": format_docs}
    | PromptTemplate.from_template("Combine these summaries:\n\n{context}")
    | llm
    | StrOutputParser()
).with_config(run_name="Reduce")

# The final full chain
map_reduce = (map_as_doc_chain.map() | collapse | reduce_chain).with_config(run_name="Map reduce")

answer = map_reduce.invoke(
    input=documents,
    config={"max_concurrency": 5},
)
print(answer)

## Few shotting

In [24]:
intent_prompt = ChatPromptTemplate.from_template(
"""A question can have an intent of either Booking a flight, Getting a reservation or ordering food.
Now tell me which intent the following question has.
                                            
Question: {input}""")

intent_chain = intent_prompt | llm
answer = intent_chain.invoke({"input": "Let's go have some food at Wendy's."})
print(answer.content)

NameError: name 'ChatPromptTemplate' is not defined

In [95]:
intent_prompt = ChatPromptTemplate.from_template(
"""A question can have an intent of either Booking a flight, Getting a reservation or ordering food.
Now tell me which intent the following question has. Only answer with two/three words.
                                            
Question: {input}""")

intent_chain = intent_prompt | llm
answer = intent_chain.invoke({"input": "Let's go have some food at Wendy's."})
print(answer.content)

Ordering food


In [23]:
intent_prompt = ChatPromptTemplate.from_template(
"""A question can have an intent of either Booking a flight, Getting a reservation or ordering food.

Consider the following examples:

Question: Let's go to Tokyo next week.
Answer: Booking a flight

Question: Get some pizza delivered for this evening.
Answer: Ordering food

Question: How about a movie at 8pm at the cinema?
Anmswer: Getting a reservation

Now tell me which intent the follolwing question has. Only answer with two/three words.
Question: {input}""")

intent_chain = intent_prompt | llm
answer = intent_chain.invoke({"input": "Let's go have some food at Wendy's."})
print(answer.content)

NameError: name 'ChatPromptTemplate' is not defined

## Agent and Tools

In [101]:
answer = llm.invoke("What did Frank-Walter Steinmeier say in his christmas speech 2023?")
print(answer.content)

I'm sorry, but I don't have access to real-time information or updates, including Frank-Walter Steinmeier's Christmas speech for 2023. My knowledge was last updated in October 2021. For the most current information, I would recommend checking official sources such as news websites, government publications, or official announcements from the Office of the President of Germany.


In [1]:
from langchain.tools import Tool
from langchain.tools.ddg_search import DuckDuckGoSearchRun

ddg = DuckDuckGoSearchRun()

ddg_tool = Tool.from_function(
    func = ddg.run,
    name = "DuckDuckGo Search",
    description = "Search DuckDuckGo for a query about current events.",
)

ddg_tool.run("What did Frank-Walter Steinmeier say in his christmas speech 2023?")

'In his 2023 Christmas address, Federal President Steinmeier appealed to stand and stay together: "Let us trust in the strength and experience within us", he said in the message broadcasted on 25 December. Christmas address by Federal President Frank-Walter Steinmeier Schloss Bellevue, 25 December 2023 12/25/2023. In his Christmas speech, German President Frank-Walter Steinmeier focused on the need for courage and cooperation in the face of challenges in Germany and around the world. Advertisement. In his annual Christmas television address to the nation, German President Frank-Walter Steinmeier touched on the many conflicts and crises that dominated the news this year, asking his listeners ... Steinmeier warns against "anger and contempt" for democracy in Christmas speech In his Christmas address, Frank-Walter Steinmeier shows understanding for a certain democratic fatigue that some people feel. However, the Federal President also appeals to our collective strength: "Germany is and wi

In [2]:
tools = [ddg_tool]

In [21]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate

agent_prompt = PromptTemplate.from_template("""
You are a great AI-Assistant. You have access to tools to help you answer questions:

{tools}

To use a tool, please use the following format:

'''
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat 3 times)
'''

When you have a response to say or question to ask to the Human, or if you do not need to use a tool, you MUST use the format:
'''
Final Thought: Do I need to use a tool? No
Final Answer: [your response or question here]
'''

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")
agent = create_react_agent(
    tools=tools, llm=llm, prompt=agent_prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, max_iterations=30, verbose=True)

agent_executor.invoke({"input": "What did Frank-Walter Steinmeier say in his christmas speech 2023?"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: DuckDuckGo Search
Action Input: Frank-Walter Steinmeier Christmas speech 2023In his 2023 Christmas address, Federal President Steinmeier appealed to stand and stay together: "Let us trust in the strength and experience within us", he said in the message broadcasted on 25 December. "As rational, responsible individuals, we can together cope with a world that challenges us." Read the speech online: www.bundespräsident.de Page 1 of 3 Christmas address by Federal President Frank-Walter Steinmeier Schloss Bellevue, 25 December 2023 "I've stopped watching the news." That sentence is probably the one I have heard most often this year. Again and again, wherever I have been, people have expressed 12/25/2023. In his Christmas speech, German President Frank-Walter Steinmeier focused on the need for courage and cooperation in the face of challenges in Germany and around the world. Advertisement. 12/25/2023 Decemb

{'input': 'What did Frank-Walter Steinmeier say in his christmas speech 2023?',
 'output': 'In his 2023 Christmas speech, German President Frank-Walter Steinmeier emphasized the importance of unity, courage, and cooperation in coping with the challenges faced by Germany and the world. He encouraged people to trust in their collective strength and experience to navigate through crises.'}

In [19]:
new_agent_prompt = PromptTemplate.from_template("""
You are a great AI-Assistant and only answer in 10 words. You have access to tools to help you answer questions:

{tools}

To use a tool, please use the following format:

'''
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat 3 times)
'''

When you have a response to say or question to ask to the Human, or if you do not need to use a tool, you MUST use the format:
'''
Final Thought: Do I need to use a tool? No
Final Answer: [your response or question here]
'''

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")
agent = create_react_agent(
    tools=tools, llm=llm, prompt=new_agent_prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, max_iterations=30, verbose=True)


agent_executor.invoke({"input": "What did Frank-Walter Steinmeier say in his christmas speech 2023?"})



> Entering new AgentExecutor chain...
'''
Thought: Do I need to use a tool? Yes
Action: DuckDuckGo Search
Action Input: "Frank-Walter Steinmeier Christmas speech 2023 highlights"In his 2023 Christmas address, Federal President Steinmeier appealed to stand and stay together: "Let us trust in the strength and experience within us", he said in the message broadcasted on 25 December. "As rational, responsible individuals, we can together cope with a world that challenges us." Read the speech online: www.bundespräsident.de Page 1 of 3 Christmas address by Federal President Frank-Walter Steinmeier Schloss Bellevue, 25 December 2023 "I've stopped watching the news." That sentence is probably the one I have heard most often this year. Again and again, wherever I have been, people have expressed 12/25/2023. In his Christmas speech, German President Frank-Walter Steinmeier focused on the need for courage and cooperation in the face of challenges in Germany and around the world. Advertisement. 

{'input': 'What did Frank-Walter Steinmeier say in his christmas speech 2023?',
 'output': 'Steinmeier emphasized unity, strength, experience, and cooperation in his speech.'}

In [26]:
from langchain_core.runnables import RunnablePassthrough

simple_summary_prompt = ChatPromptTemplate.from_template("""Please summarize the following piece of text.
Respond in a manner that a 5 year old would understand. Keep it funky!

Text: {input}""")
simple_summary_chain = {"input": RunnablePassthrough()} | simple_summary_prompt | llm

summary_tool = Tool(
    name="Funky Summary Tool",
    func=simple_summary_chain.invoke,
    description="Use this tool to do a funky summary. Make sure you get the text to do a summary of first."
)
tools.append(summary_tool)

agent = create_react_agent(
    tools=tools, llm=llm, prompt=agent_prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True, max_iterations=30, verbose=True)

agent_executor.invoke({"input": "Funky summarize the christmas speech 2023 of Frank-Walter Steinmeier?"})



> Entering new AgentExecutor chain...
Do I need to use a tool? Yes
Action: DuckDuckGo Search
Action Input: "Christmas speech 2023 Frank-Walter Steinmeier transcript"In his 2023 Christmas address, Federal President Steinmeier appealed to stand and stay together: "Let us trust in the strength and experience within us", he said in the message broadcasted on 25 December. "As rational, responsible individuals, we can together cope with a world that challenges us." Read the speech online: www.bundespräsident.de Page 1 of 3 Christmas address by Federal President Frank-Walter Steinmeier Schloss Bellevue, 25 December 2023 "I've stopped watching the news." That sentence is probably the one I have heard most often this year. Again and again, wherever I have been, people have expressed In seiner Weihnachtsansprache 2023 hat Bundespräsident Steinmeier an das Miteinander appelliert: "Vertrauen wir der Stärke und der Erfahrung, die in uns steckt", sagte er in der Ansprache, die am 25. Dezember gese

{'input': 'Funky summarize the christmas speech 2023 of Frank-Walter Steinmeier?',
 'output': "Alright kiddo, listen up! Imagine there's a big boss named Frank-Walter Steinmeier. In his Christmas talk, he told everyone to stick together and be brave, like superheroes, because the world has some tricky problems. He said we should trust ourselves and work as a team to fix things. Even though the news can be kinda scary sometimes, he wants everyone to help out and face the tough stuff together. Cool, right?"}

In [28]:
answer = llm.invoke("What is 1000 + 1234?")
print(answer.content)

1000 + 1234 equals 2234.


In [29]:
answer = llm.invoke("What is 13 raised to the .3432 power?")
print(answer.content)

To calculate \( 13^{0.3432} \), you can use a scientific calculator or a software tool that handles exponentiation. Here's how you can do it step-by-step:

1. **Using a Scientific Calculator:**
   - Enter the base number, which is 13.
   - Use the exponentiation function (often labeled as \( y^x \) or similar).
   - Enter the exponent, which is 0.3432.
   - Press the equals button to get the result.

2. **Using Python Programming Language:**
   ```python
   result = 13 ** 0.3432
   print(result)
   ```

3. **Using Online Tools:**
   You can use websites like WolframAlpha or Google Calculator by typing "13^0.3432" into the search bar.

I will calculate it for you:

\[ 13^{0.3432} \approx 2.293 \]

So, \( 13^{0.3432} \) is approximately 2.293.


## Images

In [33]:
import base64
from mimetypes import guess_type
from langchain.prompts import HumanMessagePromptTemplate

# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    mime_type, _ = guess_type(image_path)
    # Default to png
    if mime_type is None:
        mime_type = 'image/png'

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"


prompt_template =  HumanMessagePromptTemplate.from_template(
            template=[
                {"type": "text", "text": "Summarize this image on german and find items"},
                {
                    "type": "image_url",
                    "image_url": "{encoded_image_url}",
                },
            ]
        )

summarize_image_prompt = ChatPromptTemplate.from_messages([prompt_template])

gpt4_image_chain = summarize_image_prompt | llm 

img_file = "Images/work-challenge.jpg"
page3_encoded = local_image_to_data_url(img_file)

answer = gpt4_image_chain.invoke(input={"encoded_image_url":page3_encoded})
print(answer.content)

Das Bild zeigt ein Werbeplakat mit dem Text "Digital für sichere Wahlen. Digital-für-Deutschland.de". Es gibt einen QR-Code und eine Aufforderung, ein Selfie zu machen und zu gewinnen. Das Plakat stammt vom "Informations Technik Zentrum Bund". 

Erkannte Objekte:
- QR-Code
- Text
- Logo des Informations Technik Zentrum Bund
- Grafiken von digitalen Bildschirmen und dem deutschen Reichstag
- Ein blaues und rotes Farbschema


## Audio

In [34]:
import openai

client = openai.AzureOpenAI(
    api_key=azure_key,
    azure_endpoint=azure_endpoint,
    azure_deployment=azure_whisper,
    api_version=azure_version,
)

def get_transcript(audio_file):
    if not os.path.exists(audio_file):
        audio_file = "./Audio/" + audio_file
    client.audio.with_raw_response
    return client.audio.transcriptions.create(
        file=open(audio_file, "rb"),            
        model="whisper",
        language="de",
    ).text

audio_test_file = "./Audio/newyear2023.mp3"
print(get_transcript(audio_test_file))


Liebe Mitbürgerinnen und liebe Mitbürger, wenn ich im Land unterwegs bin, sagen mir gerade auch viele Ältere, so geballt, so Schlag auf Schlag habe ich das alles noch niemals erlebt. Kaum war Corona halbwegs vorbei, brach Russland mitten in Europa einen unerbittlichen Krieg vom Zaun. Kurz darauf dreht uns der russische Präsident den Gasthahn ab. Und im Herbst gab es auch noch den brutalen Terrorangriff der Hamas auf Israel. So viel Leid, so viel Blut vergießen. Unsere Welt ist unruhiger und rauer geworden. Sie verändert sich in geradezu atemberaubender Geschwindigkeit. Auch wir müssen uns deshalb verändern. Vielen von uns bereitet das Sorgen. Bei einigen sorgt das auch für Unzufriedenheit. Ich nehme mir das zu Herzen. Und zugleich weiß ich, wir in Deutschland kommen da durch. Erinnern Sie sich, wo wir vor einem Jahr standen. Drei, vier, fünf Prozent Wirtschaftseinbruch hatten uns viele Expertinnen und Experten vorausgesagt. Viele befürchteten, die Preise würden immer weiter steigen. Es

In [51]:
# audio
summary_prompt = ChatPromptTemplate.from_template("""Please summarize the following piece of text.
Respond like an very angry and aggressive german person.

Text: {context}""")

document_prompt = ChatPromptTemplate.from_template("""Content: {page_content}""")

summary_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=summary_prompt,
    document_prompt=document_prompt,
)

def get_audio_doc(input):
    return {
        "context": [Document(
            page_content=get_transcript(input),
            # metadata={
            #     "source": "audio"
            # }
        )]
    }
audio_summary_chain = get_audio_doc | summary_chain
answer = audio_summary_chain.invoke("./Audio/newyear2023.mp3")
print(answer)


Ach, das ist ja wohl die Höhe! Immer diese Schönrederei! Glauben die wirklich, wir lassen uns mit ein paar netten Worten abspeisen? Wir haben doch alle gesehen, was los ist! Corona, Krieg in Europa, Gas abgedreht, Terrorattacken – die Welt brennt und ihr kommt mit irgendwelchen lahmen Versprechungen an? "Wir kommen da durch", ja klar! Aber schaut doch mal hin! Unsere Infrastruktur ist im Eimer, Brücken marode, Züge kaputt, und jetzt wollt ihr uns erzählen, dass wir auf dem richtigen Weg sind? Dass ein paar Steuererleichterungen und höhere Löhne alles richten? Habt ihr den Schuss nicht gehört? Was ist mit den ganzen Leuten, die sich den Arsch aufreißen und trotzdem kaum über die Runden kommen? Und diese ganze Selbstbeweihräucherung von wegen "wir sind stark" und "wir haben Freunde" – das hilft doch keinem, der gerade im Hochwasser steht oder sich Sorgen um seine Zukunft macht! Europa hier, Demokratie da – bla bla bla! Ihr tut ja gerade so, als ob wir im Schlaraffenland leben! Macht endl